In [125]:
import polars as pl
from scipy.interpolate import CubicSpline

In [126]:
data = pl.read_csv("data/LandColor.csv")

In [127]:
long = (data
 .select(pl.all().exclude([".geo", "system:index"]))
 .unpivot(index = ["date", "shapeGroup", "shapeName", "shapeType"], variable_name = "variable", value_name="meanReflectance")
 .with_columns(
        pl.col("variable").str.splitn('_',2).struct.rename_fields(["landCover","band"])
    )
 .unnest("variable")
 .filter(shapeType="ADM0")
)

In [128]:
# Cubic spline from Blue Marble Next Gen
cs = CubicSpline([0,0.25,0.75,1], [0, 179, 240, 255])

In [129]:
long_term_mean = long.group_by(["shapeGroup", "shapeName", "band", "landCover"]).agg(pl.col.meanReflectance.mean()).drop_nans()

In [130]:
hexes = pl.Series("hex", list(cs((long_term_mean["meanReflectance"] * 0.0001))))

In [119]:
test = long_term_mean.with_columns(hex=hexes.cast(pl.UInt8)).select("shapeName", "landCover","band", "meanReflectance", "hex").sort(["shapeName", "landCover", "band"])
test

shapeName,landCover,band,meanReflectance,hex
str,str,str,f64,u8
"""Afghanistan""","""builtup""","""blue""",1056.069993,94
"""Afghanistan""","""builtup""","""green""",1607.366646,132
"""Afghanistan""","""builtup""","""red""",1931.855019,151
"""Afghanistan""","""crops""","""blue""",1128.281424,100
"""Afghanistan""","""crops""","""green""",1655.726108,135
…,…,…,…,…
"""Zimbabwe""","""water""","""green""",235.483749,23
"""Zimbabwe""","""water""","""red""",147.378857,15
"""Zimbabwe""","""wetland""","""blue""",278.612434,28


In [121]:
long_data = test.pivot(index=["shapeName", "landCover"], values="hex", on="band").select("shapeName", "landCover", "red", "green", "blue")

In [122]:
long_data

shapeName,landCover,red,green,blue
str,str,u8,u8,u8
"""Afghanistan""","""builtup""",151,132,94
"""Afghanistan""","""crops""",152,135,100
"""Afghanistan""","""grassland""",190,177,154
"""Afghanistan""","""savanna""",89,84,60
"""Afghanistan""","""shrubs""",171,145,106
…,…,…,…,…
"""Zimbabwe""","""grassland""",101,82,48
"""Zimbabwe""","""savanna""",77,67,36
"""Zimbabwe""","""shrubs""",86,72,41


In [123]:
# for name, data in long_data.group_by("shapeName"):  
#     country_name = name[0]
#     print(country_name)
#     for row in data.iter_rows():
#         lu_name = row[1]
#         rgb = row[2:]
#         print(lu_name)
#         print(",".join(map(str,rgb)))

countries = {}
for name, data in long_data.group_by("shapeName"):
    country_name = name[0]  
    countries[country_name] = [
        {
            "landCover": row[1],
            "rgb": ",".join(map(str, row[2:]))
        }
        for row in data.iter_rows()
    ]

In [124]:
from jinja2 import Template

# HTML Template for the palette page
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Country Color Palettes</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
        }
        .country {
            margin-bottom: 30px;
        }
        .country h2 {
            text-align: left;
        }
        .palette {
            display: flex;
            flex-wrap: wrap;
        }
        .color {
            width: 100px;
            height: 100px;
            margin: 5px;
            text-align: center;
            color: #fff;
            display: flex;
            align-items: center;
            justify-content: center;
            border-radius: 5px;
        }
    </style>
</head>
<body>
    <h1>Country Color Palettes</h1>
    {% for country, colors in countries.items() %}
    <div class="country">
        <h2>{{ country }}</h2>
        <div class="palette">
            {% for color in colors %}
            <div class="color" style="background-color: rgb({{ color.rgb }});">
                {{ color.landCover }}
            </div>
            {% endfor %}
        </div>
    </div>
    {% endfor %}
</body>
</html>
"""

# Render the HTML
template = Template(html_template)
html_content = template.render(countries=countries)

# Save the HTML to a file
output_file = "land_cover_palettes.html"
with open(output_file, "w") as f:
    f.write(html_content)

print(f"HTML file has been generated and saved as {output_file}.")


HTML file has been generated and saved as land_cover_palettes.html.
